In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np 
import time
import pyvisa as visa

In [3]:
VISA_RM = visa.ResourceManager('@py')

In [29]:
class RigolSupply():
    def __init__(self, address, n_ch, visa_resource_manager=VISA_RM):
        resource_str = f'TCPIP0::{address:s}::INSTR'
        print(resource_str)
        self.resource = VISA_RM.open_resource(resource_str, write_termination='\n', read_termination='\n')

        self.write = self.resource.write
        self.query = self.resource.query
        
        self.n_ch = n_ch
        
        self.voltages = np.zeros(n_ch)
        self.currents = np.zeros(n_ch)
        
        self.ovp_lim = np.zeros(n_ch)
        self.ocp_lim = np.zeros(n_ch)
        
        

    @property
    def IDN(self):
        return self.query("*IDN?")
    
    @property
    def IP(self):
        return self.query(":SYSTem:COMMunicate:LAN:IPADdress?")
        
    def ask(self, question):
        response = self.query(question)
        print("Question: {0:s} - Response: {1:s}".format(question, str(response)))
        return response
    
    def tell(self, statement):
        return self.write(statemnet)
    
    def enable_output(self, ch):
        return self.write(f"OUTP:STAT CH{ch},ON")
    
    def disable_output(self, ch):
        return self.write(f"OUTP:STAT CH{ch},OFF")

    def set_ocp(self, ocp):
        for ch in range(n_ch):
            self.tell(f":OUTP:OCP:VAL CH{ch},{ocp[ch]}")
    
    def get_ocp(self):
         return [self.query(f":OUTP:OCP:VAL? CH{ch}") for ch in range(self.n_ch)]
    

In [31]:
ps = RigolSupply('10.10.1.50', 2)
print(ps.IDN, ps.IP)

TCPIP0::10.10.1.50::INSTR
RIGOL TECHNOLOGIES,DP821A,DP8E203100159,00.01.14 10.10.1.50


In [33]:
supply_IPs = ["10.10.1.50", "10.10.1.51", "10.10.1.52", "10.10.1.53"]
supply_channels = [2, 2, 3, 2]
fpga_ps = [0, 2]
supplies = [RigolSupply(ip, n_ch) for ip,n_ch in zip(supply_IPs, supply_channels)]

TCPIP0::10.10.1.50::INSTR
TCPIP0::10.10.1.51::INSTR
TCPIP0::10.10.1.52::INSTR
TCPIP0::10.10.1.53::INSTR


In [36]:
for supply in supplies:
    print(supply.IDN)
    print(supply.n_ch)
    for ch in range(supply.n_ch):
        print(supply.enable_output(ch+1))
        time.sleep(2)
        print(supply.disable_output(ch+1))

RIGOL TECHNOLOGIES,DP821A,DP8E203100159,00.01.14
2
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
RIGOL TECHNOLOGIES,DP821A,DP8E203700164,00.01.14
2
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
RIGOL TECHNOLOGIES,DP832A,DP8B212500667,00.01.16
3
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
RIGOL TECHNOLOGIES,DP821A,DP8E214200286,00.01.16
2
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
(17, <StatusCode.success: 0>)
(18, <StatusCode.success: 0>)
